In [1]:
# Import dependencies
import pandas as pd
import numpy as np
from path import Path

### Import Data

In [2]:
# Read tornadoes csv into DataFrame
file_path = Path('Raw_Data/1950-2020_torn.csv')
tornadoes_df = pd.read_csv(file_path, index_col=0)
tornadoes_df

,om,yr,mo,dy,date,time,tz,st,stf,stn,...,len,wid,ns,sn,sg,f1,f2,f3,f4,fc
surr_key,,,,,,,,,,,,,,,,,,,,,
1,192,1950,10,1,10/1/1950,21:00:00,3,OK,40,23,...,15.80,10,1,1,1,25,0,0,0,0
2,193,1950,10,9,10/9/1950,2:15:00,3,NC,37,9,...,2.00,880,1,1,1,47,0,0,0,0
3,195,1950,11,20,11/20/1950,2:20:00,3,KY,21,1,...,0.10,10,1,1,1,177,0,0,0,0
4,196,1950,11,20,11/20/1950,4:00:00,3,KY,21,2,...,0.10,10,1,1,1,209,0,0,0,0
5,197,1950,11,20,11/20/1950,7:30:00,3,MS,28,14,...,2.00,37,1,1,1,101,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67500,619522,2020,9,1,9/1/2020,18:10:00,3,TX,48,0,...,0.01,30,1,1,1,191,0,0,0,0
67501,619523,2020,9,3,9/3/2020,15:57:00,3,MD,24,0,...,6.05,100,1,1,1,3,0,0,0,0
67502,619524,2020,9,5,9/5/2020,16:17:00,3,CA,6,0,...,12.02,50,1,1,1,39,0,0,0,0


In [3]:
tornadoes_df.columns

Index(['om', 'yr', 'mo', 'dy', 'date', 'time', 'tz', 'st', 'stf', 'stn', 'f',
       'inj', 'fat', 'loss', 'closs', 'slat', 'slon', 'elat', 'elon', 'len',
       'wid', 'ns', 'sn', 'sg', 'f1', 'f2', 'f3', 'f4', 'fc'],
      dtype='object')

In [4]:
tornadoes_df.dtypes

om         int64
yr         int64
mo         int64
dy         int64
date      object
time      object
tz         int64
st        object
stf        int64
stn        int64
f          int64
inj        int64
fat        int64
loss     float64
closs    float64
slat     float64
slon     float64
elat     float64
elon     float64
len      float64
wid        int64
ns         int64
sn         int64
sg         int64
f1         int64
f2         int64
f3         int64
f4         int64
fc         int64
dtype: object

In [5]:
# Rename Columns
tornadoes_df.rename(columns = {"stf":"st_fips",'f':'ef'}, inplace = True)

In [6]:
# Rearrange colums
cols = ['yr', 'mo', 'dy', 'date', 'time', 'st', 'st_fips', 'f1', 'f2', 'f3', 'f4', 
        'ef', 'inj', 'fat', 'loss', 'closs', 'slat', 'slon', 'elat', 'elon', 'len', 'wid' ]

tornadoes_df[cols]

,yr,mo,dy,date,time,st,st_fips,f1,f2,f3,...,inj,fat,loss,closs,slat,slon,elat,elon,len,wid
surr_key,,,,,,,,,,,,,,,,,,,,,
1,1950,10,1,10/1/1950,21:00:00,OK,40,25,0,0,...,0,0,4.0,0.0,36.7300,-102.5200,36.8800,-102.3000,15.80,10
2,1950,10,9,10/9/1950,2:15:00,NC,37,47,0,0,...,3,0,5.0,0.0,34.1700,-78.6000,0.0000,0.0000,2.00,880
3,1950,11,20,11/20/1950,2:20:00,KY,21,177,0,0,...,0,0,5.0,0.0,37.3700,-87.2000,0.0000,0.0000,0.10,10
4,1950,11,20,11/20/1950,4:00:00,KY,21,209,0,0,...,0,0,5.0,0.0,38.2000,-84.5000,0.0000,0.0000,0.10,10
5,1950,11,20,11/20/1950,7:30:00,MS,28,101,0,0,...,3,0,4.0,0.0,32.4200,-89.1300,0.0000,0.0000,2.00,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67500,2020,9,1,9/1/2020,18:10:00,TX,48,191,0,0,...,0,0,0.0,0.0,34.5706,-100.8394,34.5706,-100.8394,0.01,30
67501,2020,9,3,9/3/2020,15:57:00,MD,24,3,0,0,...,0,0,1250000.0,0.0,38.9480,-76.5695,38.9233,-76.4615,6.05,100
67502,2020,9,5,9/5/2020,16:17:00,CA,6,39,0,0,...,0,0,0.0,0.0,37.3500,-119.3400,37.5218,-119.3056,12.02,50


In [7]:
# Sort DF by Year then State
tornadoes_df.sort_values(by=['yr','mo','st'])

,om,yr,mo,dy,date,time,tz,st,st_fips,stn,...,len,wid,ns,sn,sg,f1,f2,f3,f4,fc
surr_key,,,,,,,,,,,,,,,,,,,,,
11,4,1950,1,13,1/13/1950,5:25:00,3,AR,5,1,...,0.60,17,1,1,1,113,0,0,0,0
13,6,1950,1,25,1/25/1950,21:00:00,3,IL,17,3,...,0.10,100,1,1,1,91,0,0,0,0
17,1,1950,1,3,1/3/1950,11:10:00,3,IL,17,1,...,3.30,100,2,1,2,119,0,0,0,0
18,2,1950,1,3,1/3/1950,11:55:00,3,IL,17,2,...,3.60,130,1,1,1,135,0,0,0,0
12,5,1950,1,25,1/25/1950,19:30:00,3,MO,29,2,...,2.30,300,1,1,1,93,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66438,619601,2020,12,23,12/23/2020,15:54:00,3,TX,48,0,...,1.23,20,1,1,1,351,0,0,0,0
66439,619602,2020,12,23,12/23/2020,18:59:00,3,TX,48,0,...,0.39,45,1,1,1,241,0,0,0,0
66460,619623,2020,12,30,12/30/2020,10:41:00,3,TX,48,0,...,0.98,75,1,1,1,349,0,0,0,0


### Clean the tor_EF dataframe

### Clean the whole dataset

In [8]:
# Filter out any bad data (ie 0<=EF<=5)
tornadoes_clean = tornadoes_df[ (tornadoes_df['ef'] >= 0) & ( tornadoes_df["ef"] <= 5)]
tornadoes_clean

,om,yr,mo,dy,date,time,tz,st,st_fips,stn,...,len,wid,ns,sn,sg,f1,f2,f3,f4,fc
surr_key,,,,,,,,,,,,,,,,,,,,,
1,192,1950,10,1,10/1/1950,21:00:00,3,OK,40,23,...,15.80,10,1,1,1,25,0,0,0,0
2,193,1950,10,9,10/9/1950,2:15:00,3,NC,37,9,...,2.00,880,1,1,1,47,0,0,0,0
3,195,1950,11,20,11/20/1950,2:20:00,3,KY,21,1,...,0.10,10,1,1,1,177,0,0,0,0
4,196,1950,11,20,11/20/1950,4:00:00,3,KY,21,2,...,0.10,10,1,1,1,209,0,0,0,0
5,197,1950,11,20,11/20/1950,7:30:00,3,MS,28,14,...,2.00,37,1,1,1,101,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67499,619521,2020,9,1,9/1/2020,16:24:00,3,TN,47,0,...,2.10,50,1,1,1,95,0,0,0,0
67501,619523,2020,9,3,9/3/2020,15:57:00,3,MD,24,0,...,6.05,100,1,1,1,3,0,0,0,0
67502,619524,2020,9,5,9/5/2020,16:17:00,3,CA,6,0,...,12.02,50,1,1,1,39,0,0,0,0


In [9]:
# Filter for Lower 48 (exclude PR, VI, HI, AK)
tornadoes_clean = tornadoes_clean[tornadoes_clean['st'] != 'PR']
tornadoes_clean = tornadoes_clean[tornadoes_clean['st'] != 'VI']
tornadoes_clean = tornadoes_clean[tornadoes_clean['st'] != 'HI']
tornadoes_clean = tornadoes_clean[tornadoes_clean['st'] != 'AK']
tornadoes_clean

,om,yr,mo,dy,date,time,tz,st,st_fips,stn,...,len,wid,ns,sn,sg,f1,f2,f3,f4,fc
surr_key,,,,,,,,,,,,,,,,,,,,,
1,192,1950,10,1,10/1/1950,21:00:00,3,OK,40,23,...,15.80,10,1,1,1,25,0,0,0,0
2,193,1950,10,9,10/9/1950,2:15:00,3,NC,37,9,...,2.00,880,1,1,1,47,0,0,0,0
3,195,1950,11,20,11/20/1950,2:20:00,3,KY,21,1,...,0.10,10,1,1,1,177,0,0,0,0
4,196,1950,11,20,11/20/1950,4:00:00,3,KY,21,2,...,0.10,10,1,1,1,209,0,0,0,0
5,197,1950,11,20,11/20/1950,7:30:00,3,MS,28,14,...,2.00,37,1,1,1,101,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67499,619521,2020,9,1,9/1/2020,16:24:00,3,TN,47,0,...,2.10,50,1,1,1,95,0,0,0,0
67501,619523,2020,9,3,9/3/2020,15:57:00,3,MD,24,0,...,6.05,100,1,1,1,3,0,0,0,0
67502,619524,2020,9,5,9/5/2020,16:17:00,3,CA,6,0,...,12.02,50,1,1,1,39,0,0,0,0


In [10]:
# Investigate closs
tornadoes_clean["closs"].value_counts()

0.000       66144
0.010          88
0.005          75
0.001          49
0.002          45
            ...  
0.018           1
1.900           1
1.050           1
0.800           1
1500.000        1
Name: closs, Length: 144, dtype: int64

In [11]:
# Replace loss and closs 0 with NaN ( 0 means no data avaliable NOT $0 in loss)
tornadoes_clean['closs']=tornadoes_clean['closs'].replace(0, np.nan)
tornadoes_clean['loss']=tornadoes_clean['loss'].replace(0, np.nan)
tornadoes_clean

,om,yr,mo,dy,date,time,tz,st,st_fips,stn,...,len,wid,ns,sn,sg,f1,f2,f3,f4,fc
surr_key,,,,,,,,,,,,,,,,,,,,,
1,192,1950,10,1,10/1/1950,21:00:00,3,OK,40,23,...,15.80,10,1,1,1,25,0,0,0,0
2,193,1950,10,9,10/9/1950,2:15:00,3,NC,37,9,...,2.00,880,1,1,1,47,0,0,0,0
3,195,1950,11,20,11/20/1950,2:20:00,3,KY,21,1,...,0.10,10,1,1,1,177,0,0,0,0
4,196,1950,11,20,11/20/1950,4:00:00,3,KY,21,2,...,0.10,10,1,1,1,209,0,0,0,0
5,197,1950,11,20,11/20/1950,7:30:00,3,MS,28,14,...,2.00,37,1,1,1,101,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67499,619521,2020,9,1,9/1/2020,16:24:00,3,TN,47,0,...,2.10,50,1,1,1,95,0,0,0,0
67501,619523,2020,9,3,9/3/2020,15:57:00,3,MD,24,0,...,6.05,100,1,1,1,3,0,0,0,0
67502,619524,2020,9,5,9/5/2020,16:17:00,3,CA,6,0,...,12.02,50,1,1,1,39,0,0,0,0


In [12]:
# Starting in 2016 the loss data is in whole dollar amounts need to convert rows with years prior to 2016 to whole dollar
# original loss and closs were in millions of dollars
# Multiply loss and closs by $1,000,000 to get whole number values
loss = tornadoes_clean.loc[tornadoes_clean["yr"]<2016]
loss

,om,yr,mo,dy,date,time,tz,st,st_fips,stn,...,len,wid,ns,sn,sg,f1,f2,f3,f4,fc
surr_key,,,,,,,,,,,,,,,,,,,,,
1,192,1950,10,1,10/1/1950,21:00:00,3,OK,40,23,...,15.80,10,1,1,1,25,0,0,0,0
2,193,1950,10,9,10/9/1950,2:15:00,3,NC,37,9,...,2.00,880,1,1,1,47,0,0,0,0
3,195,1950,11,20,11/20/1950,2:20:00,3,KY,21,1,...,0.10,10,1,1,1,177,0,0,0,0
4,196,1950,11,20,11/20/1950,4:00:00,3,KY,21,2,...,0.10,10,1,1,1,209,0,0,0,0
5,197,1950,11,20,11/20/1950,7:30:00,3,MS,28,14,...,2.00,37,1,1,1,101,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61213,601870,2015,9,26,9/26/2015,14:48:00,3,TX,48,0,...,0.36,50,1,1,1,249,0,0,0,0
61214,598567,2015,9,29,9/29/2015,8:00:00,3,FL,12,0,...,0.06,10,1,1,1,21,0,0,0,0
61215,599512,2015,9,29,9/29/2015,20:02:00,3,MD,24,0,...,4.40,75,1,1,1,33,27,0,0,0


In [13]:
loss["loss"] = loss["loss"]*1000000
loss["closs"] = loss["closs"]*1000000
loss

C:\Users\Nat\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Nat\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,om,yr,mo,dy,date,time,tz,st,st_fips,stn,...,len,wid,ns,sn,sg,f1,f2,f3,f4,fc
surr_key,,,,,,,,,,,,,,,,,,,,,
1,192,1950,10,1,10/1/1950,21:00:00,3,OK,40,23,...,15.80,10,1,1,1,25,0,0,0,0
2,193,1950,10,9,10/9/1950,2:15:00,3,NC,37,9,...,2.00,880,1,1,1,47,0,0,0,0
3,195,1950,11,20,11/20/1950,2:20:00,3,KY,21,1,...,0.10,10,1,1,1,177,0,0,0,0
4,196,1950,11,20,11/20/1950,4:00:00,3,KY,21,2,...,0.10,10,1,1,1,209,0,0,0,0
5,197,1950,11,20,11/20/1950,7:30:00,3,MS,28,14,...,2.00,37,1,1,1,101,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61213,601870,2015,9,26,9/26/2015,14:48:00,3,TX,48,0,...,0.36,50,1,1,1,249,0,0,0,0
61214,598567,2015,9,29,9/29/2015,8:00:00,3,FL,12,0,...,0.06,10,1,1,1,21,0,0,0,0
61215,599512,2015,9,29,9/29/2015,20:02:00,3,MD,24,0,...,4.40,75,1,1,1,33,27,0,0,0


In [14]:
# drop all rows in tor_EF between 2008-2015
tornadoes_clean_2016_2020 = tornadoes_clean[tornadoes_clean["yr"] >= 2016]
tornadoes_clean_2016_2020

,om,yr,mo,dy,date,time,tz,st,st_fips,stn,...,len,wid,ns,sn,sg,f1,f2,f3,f4,fc
surr_key,,,,,,,,,,,,,,,,,,,,,
61218,614398,2016,10,12,10/12/2016,16:26:00,3,IN,18,0,...,1.25,30,1,1,1,141,0,0,0,0
61219,614399,2016,10,12,10/12/2016,16:30:00,3,IN,18,0,...,0.05,40,1,1,1,141,0,0,0,0
61220,614400,2016,10,14,10/14/2016,9:18:00,3,OR,41,0,...,0.70,225,1,1,1,57,0,0,0,0
61222,614382,2016,10,4,10/4/2016,16:32:00,3,KS,20,0,...,0.10,40,1,1,1,143,0,0,0,0
61223,614383,2016,10,4,10/4/2016,17:27:00,3,KS,20,0,...,4.92,75,1,1,1,27,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67499,619521,2020,9,1,9/1/2020,16:24:00,3,TN,47,0,...,2.10,50,1,1,1,95,0,0,0,0
67501,619523,2020,9,3,9/3/2020,15:57:00,3,MD,24,0,...,6.05,100,1,1,1,3,0,0,0,0
67502,619524,2020,9,5,9/5/2020,16:17:00,3,CA,6,0,...,12.02,50,1,1,1,39,0,0,0,0


In [15]:
tornadoes_clean = loss.append(tornadoes_clean_2016_2020)
tornadoes_clean

,om,yr,mo,dy,date,time,tz,st,st_fips,stn,...,len,wid,ns,sn,sg,f1,f2,f3,f4,fc
surr_key,,,,,,,,,,,,,,,,,,,,,
1,192,1950,10,1,10/1/1950,21:00:00,3,OK,40,23,...,15.80,10,1,1,1,25,0,0,0,0
2,193,1950,10,9,10/9/1950,2:15:00,3,NC,37,9,...,2.00,880,1,1,1,47,0,0,0,0
3,195,1950,11,20,11/20/1950,2:20:00,3,KY,21,1,...,0.10,10,1,1,1,177,0,0,0,0
4,196,1950,11,20,11/20/1950,4:00:00,3,KY,21,2,...,0.10,10,1,1,1,209,0,0,0,0
5,197,1950,11,20,11/20/1950,7:30:00,3,MS,28,14,...,2.00,37,1,1,1,101,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67499,619521,2020,9,1,9/1/2020,16:24:00,3,TN,47,0,...,2.10,50,1,1,1,95,0,0,0,0
67501,619523,2020,9,3,9/3/2020,15:57:00,3,MD,24,0,...,6.05,100,1,1,1,3,0,0,0,0
67502,619524,2020,9,5,9/5/2020,16:17:00,3,CA,6,0,...,12.02,50,1,1,1,39,0,0,0,0


In [16]:
tornadoes_clean.columns

Index(['om', 'yr', 'mo', 'dy', 'date', 'time', 'tz', 'st', 'st_fips', 'stn',
       'ef', 'inj', 'fat', 'loss', 'closs', 'slat', 'slon', 'elat', 'elon',
       'len', 'wid', 'ns', 'sn', 'sg', 'f1', 'f2', 'f3', 'f4', 'fc'],
      dtype='object')

### Filter cleaned data for for years needed 2008-2020 (EF scale tornado ratings)

In [17]:
# Delete columns we dont need
tornadoes_df_clean = tornadoes_clean.drop(["om","tz","stn", 'ns', 'sn', 'sg', 'f1', 'f2', 'f3', 'f4', 'fc'], axis =1)

In [18]:
# 1950-2020
tornadoes_df_clean.to_csv("Cleaned_Data/1950-2020_tornadoes_cleaned.csv")

In [19]:
# Filter for years 1950-2007 --> F Scale
tornadoes_F = tornadoes_df_clean[tornadoes_df['yr'] < 2008]
tornadoes_F

C:\Users\Nat\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,yr,mo,dy,date,time,st,st_fips,ef,inj,fat,loss,closs,slat,slon,elat,elon,len,wid
surr_key,,,,,,,,,,,,,,,,,,
1,1950,10,1,10/1/1950,21:00:00,OK,40,1,0,0,4000000.0,NaN,36.73,-102.52,36.88,-102.30,15.80,10
2,1950,10,9,10/9/1950,2:15:00,NC,37,3,3,0,5000000.0,NaN,34.17,-78.60,0.00,0.00,2.00,880
3,1950,11,20,11/20/1950,2:20:00,KY,21,2,0,0,5000000.0,NaN,37.37,-87.20,0.00,0.00,0.10,10
4,1950,11,20,11/20/1950,4:00:00,KY,21,1,0,0,5000000.0,NaN,38.20,-84.50,0.00,0.00,0.10,10
5,1950,11,20,11/20/1950,7:30:00,MS,28,1,3,0,4000000.0,NaN,32.42,-89.13,0.00,0.00,2.00,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51188,2007,9,6,9/6/2007,13:45:00,NM,35,0,0,0,NaN,NaN,32.65,-108.50,32.61,-108.46,3.05,40
51189,2007,9,6,9/6/2007,16:27:00,NE,31,0,0,0,NaN,NaN,41.69,-96.50,41.73,-96.48,3.16,50
51190,2007,9,6,9/6/2007,17:28:00,IL,17,0,0,0,NaN,NaN,39.95,-89.57,39.95,-89.57,0.10,10


In [20]:
# Export Cleaned Data to CSV
tornadoes_F.to_csv("Cleaned_Data/1950-2007_tornadoes_cleaned.csv")

In [21]:
# Filter for years 2008-2020 --> EF Scale
tornadoes_EF = tornadoes_df_clean[tornadoes_df['yr'] >= 2008]
tornadoes_EF

C:\Users\Nat\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,yr,mo,dy,date,time,st,st_fips,ef,inj,fat,loss,closs,slat,slon,elat,elon,len,wid
surr_key,,,,,,,,,,,,,,,,,,
51193,2008,10,11,10/11/2008,18:33:00,NM,35,0,0,0,12000.0,NaN,35.1500,-105.9400,0.0000,0.0000,0.02,10
51194,2008,10,11,10/11/2008,21:54:00,KS,20,0,0,0,NaN,NaN,39.3300,-101.5500,0.0000,0.0000,0.25,10
51195,2008,10,24,10/24/2008,16:55:00,GA,13,0,0,0,1000.0,NaN,31.0200,-81.8100,0.0000,0.0000,0.25,25
51196,2008,10,6,10/6/2008,13:46:00,AR,5,1,0,0,75000.0,NaN,34.5800,-94.1500,34.6300,-94.1200,3.41,250
51197,2008,10,6,10/6/2008,16:05:00,TX,48,0,0,0,75000.0,NaN,33.0700,-94.1000,33.0900,-94.0800,1.50,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67499,2020,9,1,9/1/2020,16:24:00,TN,47,0,0,0,NaN,2000.0,36.3290,-89.4981,36.3517,-89.4729,2.10,50
67501,2020,9,3,9/3/2020,15:57:00,MD,24,0,0,0,1250000.0,NaN,38.9480,-76.5695,38.9233,-76.4615,6.05,100
67502,2020,9,5,9/5/2020,16:17:00,CA,6,2,0,0,NaN,NaN,37.3500,-119.3400,37.5218,-119.3056,12.02,50


In [22]:
# Export Cleaned Data to CSV
tornadoes_EF.to_csv("Cleaned_Data/2008-2020_tornadoes_cleaned.csv")

In [23]:
# Summary Stats on Data
tornadoes_EF[["ef","inj","fat","loss","closs","len","wid"]].describe()

,ef,inj,fat,loss,closs,len,wid
count,15891.000000,15891.000000,15891.000000,8.569000e+03,8.280000e+02,15891.000000,15891.000000
mean,0.627903,0.993330,0.092379,3.518636e+06,2.506938e+05,3.757608,174.618526
std,0.807329,17.614538,1.787981,5.895398e+07,1.410030e+06,6.880436,276.586036
min,0.000000,0.000000,0.000000,1.000000e+02,1.000000e+02,0.000000,0.000000
25%,0.000000,0.000000,0.000000,2.000000e+04,5.000000e+03,0.400000,50.000000
50%,0.000000,0.000000,0.000000,5.400000e+04,1.200000e+04,1.520000,75.000000
75%,1.000000,0.000000,0.000000,2.500000e+05,1.000000e+05,4.345000,200.000000
max,5.000000,1500.000000,158.000000,2.800100e+09,2.352000e+07,148.970000,4576.000000
